In [1]:
import pandas as pd
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np



import scipy.stats as stats

from sklearn.neighbors import KernelDensity

from datetime import datetime, timedelta


import numpy as np
from scipy import stats
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt



import numpy as np
import ot

import requests
from bs4 import BeautifulSoup
import json


import numpy as np
import pandas as pd
import ot

In [11]:
def implied(odds):
    if odds < 0:
        return -odds/(-odds+100)
    else:
        return 100/(odds + 100)
    
    
def kelley(odds, bank, p):
    if odds < 0:
        b = (100)/(-odds)
    else:
        b = (odds)/100
    return (p - ((1-p)/b))*bank


def get_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error")
        return None

def convert_utc_to_chicago(utc_time_str):
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%MZ')
    chicago_offset = timedelta(hours=-8)
    chicago_time = utc_time + chicago_offset
    return chicago_time.strftime('%Y-%m-%d %H:%M:%S')


csv = pd.read_csv("team_id.csv")

def get_id(name, csv = csv):
    res = csv[csv['name'] == name]
    return res


def get_game_ids(team_id):
    url = f"https://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/teams/{team_id}/events?limit=100"
    df = get_data(url)['items']
    game_id = []
    for game in df:
        game_id.append(get_data(game['$ref'])['id'])
    
    return game_id


def win_prob(game_id):
    
    win_prob_url = f"https://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/events/{game_id}/competitions/{game_id}/probabilities?limit=1000"
    win_data = get_data(win_prob_url)
    
    away_url = win_data['items'][-1]['awayTeam']['$ref']
    home_url = win_data['items'][-1]['homeTeam']['$ref']

    home_name = requests.get(home_url).json()['location']
    away_name = requests.get(away_url).json()['location']
    
    home_p = win_data['items'][-1]['homeWinPercentage']
    away_p = win_data['items'][-1]['awayWinPercentage']
    time = win_data['items'][-1]['lastModified']
    
    print(f"{convert_utc_to_chicago(time)}")
    print(f"Home: {home_name}: {home_p}")
    print(f"Away: {away_name}: {away_p}")
    
    return home_p, away_p, home_name, away_name



def player_boxscore(name, game_id):
    boxscore_url = f"https://cdn.espn.com/core/mens-college-basketball/boxscore?xhr=1&gameId={game_id}"
    box_data = get_data(boxscore_url)
    
    name1 = box_data['gamepackageJSON']['boxscore']['players'][0]['team']['location']
    name2 = box_data['gamepackageJSON']['boxscore']['players'][1]['team']['location']
    
    col_names = ['Name'] + box_data['gamepackageJSON']['boxscore']['players'][0]['statistics'][0]['names']
    i = 0
    if name == name1:
        i = 0
    elif name == name2:
        i = 1
    else:
        print("Error in Name")
    
    player1 = box_data['gamepackageJSON']['boxscore']['players'][i]['statistics'][0]['athletes']
    rows1 = []
    for stat1 in player1:
        s1 = stat1['stats']
        if s1:
            rows1.append([stat1['athlete']['displayName']] + s1)

    df1 = pd.DataFrame(rows1, columns = col_names)
    
    return df1



def get_starters(name, game_id):
    boxscore_url = f"https://cdn.espn.com/core/mens-college-basketball/boxscore?xhr=1&gameId={game_id}"
    box_data = get_data(boxscore_url)
    
    name1 = box_data['gamepackageJSON']['boxscore']['players'][0]['team']['location']
    name2 = box_data['gamepackageJSON']['boxscore']['players'][1]['team']['location']

    col_names = ['Name'] + box_data['gamepackageJSON']['boxscore']['players'][0]['statistics'][0]['names']
    i = 0
    if name == name1:
        i = 0
    elif name == name2:
        i = 1
    else:
        print("Error in Name")
        
    player1 = box_data['gamepackageJSON']['boxscore']['players'][i]['statistics'][0]['athletes']
    rows1 = []
    for stat1 in player1:
        starter1 = stat1['starter']
        if starter1:
            rows1.append(stat1['athlete']['displayName'])
    
    return rows1


def all_games(name, team_id):
    game_ids = get_game_ids(team_id)
    del game_ids[-1]
    df = pd.DataFrame(columns = ['game_id', 'Name', 'MIN', 'FG', '3PT', 'FT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS'])
    for game in game_ids:
        sub = player_boxscore(name, game)
        sub['game_id'] = game
        df = pd.concat([df,sub])
    print("Done")
    return df


def last_games(name, team_id):
    game_ids = get_game_ids(team_id)
    del game_ids[-1]
    df = pd.DataFrame(columns = ['game_id', 'Name', 'MIN', 'FG', '3PT', 'FT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS'])
    for game in game_ids[-5:]:
        sub = player_boxscore(name, game)
        sub['game_id'] = game
        df = pd.concat([df,sub])
    print("Done")
    return df



def fitting(data):
    # Use grid search cross-validation to optimize the bandwidth
    params = {'bandwidth': np.logspace(-1, 1, 50)}
    grid = GridSearchCV(KernelDensity(), params)
    grid.fit(data)  # No need to reshape data for multivariate KDE

    print("Best bandwidth: ", grid.best_estimator_.bandwidth)

    return grid.best_estimator_



from sklearn.utils import check_X_y

def fitting_weights(data, weights):
    # Check the data and weights have compatible shapes
#     data, weights = check_X_y(data, weights, accept_sparse=True, y_numeric=True, ensure_2d=False, dtype=None)
    
    # Use grid search cross-validation to optimize the bandwidth
    params = {'bandwidth': np.logspace(-1, 1, 50)}
    grid = GridSearchCV(KernelDensity(), params)
    
    # Pass the weights to the fit method
    grid.fit(data, sample_weight=weights)

    print("Best bandwidth: ", grid.best_estimator_.bandwidth)

    return grid.best_estimator_


def estimate_probability1(kde, x, column_index, num_samples=10000):
    # Generate samples from the fitted KDE
    samples = kde.sample(num_samples)

    # Compute the empirical CDF at x for the specified column
    cdf_x = np.mean(samples[:,column_index] > x)

    return cdf_x



def estimate_joint_probability1(kde, x_values, column_indices, num_samples=10000):
    # Generate samples from the fitted KDE
    samples = kde.sample(num_samples)

    # Compute the empirical joint CDF at the given values for the specified columns
    joint_cdf_x = np.mean(np.all(samples[:, column_indices] > x_values, axis=1))

    return joint_cdf_x



def estimate_probability(kde, x, column_index, num_samples=10000, num_bootstrap_samples=100):
    # Generate samples from the fitted KDE
    # samples = kde.sample(num_samples)

    # Compute the empirical CDF at x for the specified column
    cdf_x_samples = np.array([np.mean(kde.sample(num_samples)[:,column_index] > x) for _ in range(num_bootstrap_samples)])

    # Compute the mean and standard deviation of the bootstrap samples
    mean_cdf_x = np.mean(cdf_x_samples)
    std_cdf_x = np.std(cdf_x_samples)

    # Compute the 95% confidence interval
    ci_lower = mean_cdf_x - 1.96 * std_cdf_x
    ci_upper = mean_cdf_x + 1.96 * std_cdf_x

    return mean_cdf_x, ci_lower, ci_upper


# create over under functionality

def estimate_joint_probability(kde, x_values, column_indices, over_under, num_samples=10000, num_bootstrap_samples=100):
    # Generate samples from the fitted KDE
    # samples = kde.sample(num_samples)
    
    # Compute the empirical joint CDF at the given values for the specified columns

    if over_under == (1,1):
        joint_cdf_x_samples = np.array([np.mean(np.all(kde.sample(num_samples)[:, column_indices] > x_values, axis=1)) for _ in range(num_bootstrap_samples)])
    
    elif over_under == (0,0):
        joint_cdf_x_samples = np.array([np.mean(np.all(kde.sample(num_samples)[:, column_indices] < x_values, axis=1)) for _ in range(num_bootstrap_samples)])
    
    elif over_under == (0,1):
        joint_cdf_x_samples = []
        for _ in range(num_bootstrap_samples):
            kk = kde.sample(num_samples)
            joint_cdf_x_samples.append(np.mean(np.all([kk[:, column_indices[0]] < x_values[0], 
                                                        kk[:, column_indices[1]] > x_values[1]], axis=0)))

    elif over_under == (1,0):
        joint_cdf_x_samples = []
        for _ in range(num_bootstrap_samples):
            kk = kde.sample(num_samples)
            joint_cdf_x_samples.append(np.mean(np.all([kk[:, column_indices[0]] > x_values[0], 
                                                        kk[:, column_indices[1]] < x_values[1]], axis=0)))
            
    joint_cdf_x_samples = np.array(joint_cdf_x_samples)
    
    # Compute the mean and standard deviation of the bootstrap samples
    mean_joint_cdf_x = np.mean(joint_cdf_x_samples)
    std_joint_cdf_x = np.std(joint_cdf_x_samples)

    # Compute the 95% confidence interval
    ci_lower = mean_joint_cdf_x - 1.96 * std_joint_cdf_x
    ci_upper = mean_joint_cdf_x + 1.96 * std_joint_cdf_x

    return mean_joint_cdf_x, ci_lower, ci_upper



def game_projection(game_id):
    projection_url = f"https://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/events/{game_id}/competitions/{game_id}/predictor"
    proj = get_data(projection_url)
    
    away_url = proj['awayTeam']['team']['$ref']
    home_url = proj['homeTeam']['team']['$ref']

    home_name = requests.get(home_url).json()['location']
    away_name = requests.get(away_url).json()['location']
    
    data1 = proj['homeTeam']['statistics']
    data2 = proj['awayTeam']['statistics']
    
    col = []
    rows1 = []
    rows2 = []
    
    for d1, d2 in zip(data1, data2):
        col.append(d1['name'])
        rows1.append(d1['displayValue'])
        rows2.append(d2['displayValue'])
    
    
    df1 = pd.DataFrame([rows1], columns = col)
    df2 = pd.DataFrame([rows2], columns = col)
    
    return df1, df2


def team_boxscore(name, game_id):
    boxscore_url = f"https://cdn.espn.com/core/mens-college-basketball/boxscore?xhr=1&gameId={game_id}"
    box_data = get_data(boxscore_url)
    teams = box_data['gamepackageJSON']['boxscore']['teams']

    name1 = teams[0]['team']['location']
    name2 = teams[1]['team']['location']

    i = 0
    if name == name1:
        i = 0
    elif name == name2:
        i = 1
    else:
        print("Error in Name")

    dat1 = [game_id,teams[i]['team']['location']]
    col_names = ['game_id','Team']

    for stat1 in teams[i]['statistics']:
        col_names.append(stat1['label'])
        dat1.append(stat1['displayValue'])

    df = pd.DataFrame([dat1], columns=col_names)

    return df


def past_team_boxscore(name, team_id):
    game_ids = get_game_ids(team_id)
    del game_ids[-1]
    df = pd.DataFrame()
    for game in game_ids:
        sub = team_boxscore(name, game)
        df = pd.concat([df,sub])
    print("Done")
    return df




## Optimal Transport finding weights


def get_player_stats(team_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f'https://www.espn.com/mens-college-basketball/team/stats/_/id/{team_id}'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    script = soup.find('script', text=lambda t: t and 'playerStats' in t)

    json_str = script.string.split('playerStats":')[1].split('}]]')[0] + '}]]'

    player_stats = json.loads(json_str)[0]


    col_names = []

    for nn in player_stats[0]['statGroups']['stats']:
            col_names.append(nn['abbreviation'])

    df = pd.DataFrame(columns = col_names)

    for ax in player_stats:
        sub = []
        for stats in ax['statGroups']['stats']:
            sub.append(stats['value'])
        df.loc[len(df)] = sub

    for column in df.columns[1:]:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    df.fillna(0,inplace =True)
    return df


def get_opponent_id(team_id):
    url = f"https://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/teams/{team_id}/events?limit=100"
    df = get_data(url)['items']
    opp_id = []
    for dd in df:
        for comp in get_data(dd['$ref'])['competitions'][0]['competitors']:
            if comp['id'] != team_id:
                opp_id.append(comp['id'])
                break
    return opp_id[:-1]


def set_up_data(team_id):
    home_stats = get_player_stats(team_id)
    opp_id = get_opponent_id(team_id)
    ls = []
    for opp in opp_id:
        ls.append(get_player_stats(opp))
    return home_stats, ls



def calculate_unbalanced_wasserstein(reference_team_df, other_teams_dfs, reg_e=0.1, reg_m=0.1):
    # Convert the reference team dataframe to a numpy array
    reference_team_array = reference_team_df.to_numpy()

    # Initialize a dictionary to store the Wasserstein distances
    distances = {}

    # Iterate over the list of other teams' dataframes
    for i, team_df in enumerate(other_teams_dfs):
        # Ensure the team dataframe has the same stats as columns in the same order
        team_df = team_df[reference_team_df.columns]

        # Convert the team dataframe to a numpy array
        team_array = team_df.to_numpy()

        # Initialize a dictionary to store the Wasserstein distances for each stat
        distances[i] = {}

        # Iterate over each statistic (column)
        for stat_idx in range(reference_team_array.shape[1]):
            # Extract the distributions for the current stat
            ref_stat_distribution = reference_team_array[:, stat_idx]
            team_stat_distribution = team_array[:, stat_idx]

            # Compute the cost matrix for the Wasserstein distance calculation
            cost_matrix = ot.dist(ref_stat_distribution.reshape(-1, 1), team_stat_distribution.reshape(-1, 1))

            epsilon = 1e-8
            # 'a' and 'b' are uniform distributions if you're not normalizing
            a = np.ones((len(ref_stat_distribution),)) / len(ref_stat_distribution)
            b = np.ones((len(team_stat_distribution),)) / len(team_stat_distribution)

            # Define the range for the hyperparameters
            reg_e_values = np.linspace(0.01, 1, 10)
            reg_m_values = np.linspace(0.01, 1, 10)

            # Initialize the best score and the best parameters
            best_score = float('inf')
            best_params = None

            # Perform random search over the hyperparameters
            for reg_e in reg_e_values:
                for reg_m in reg_m_values:
                    # Compute the unbalanced Wasserstein distance with the current parameters
                    wasserstein_distance = ot.unbalanced.sinkhorn_unbalanced(
                        a, b, cost_matrix, reg_e, reg_m
                    )[0]
                    
                    # If the current score is better than the best score, update the best score and the best parameters
                    if np.all(wasserstein_distance < best_score): 
                        best_score = wasserstein_distance
                        best_params = (reg_e, reg_m)
#             print(best_params)
            # Store the best Wasserstein distance for the current stat
            distances[i][stat_idx] = best_score

    weights = np.array([1, 1] + list(range(9, 0, -1)))
    weights = weights / weights.sum()  # Normalize the weights

    # Calculate the weighted average distance for each team
    weighted_average_distances = {
        team: np.dot(np.array([np.linalg.norm(v) for v in stats.values()]), weights)
        for team, stats in distances.items()
    }

    # Normalize the weighted average distances
    max_weighted_average_distance = max(weighted_average_distances.values())
    normalized_weighted_average_distances = {
        team: dist / max_weighted_average_distance
        for team, dist in weighted_average_distances.items()
    }

    # Convert normalized weighted average distances to similarity scores
    similarity_weights = {
        team: normalized_weighted_average_distances[team]
        for team in normalized_weighted_average_distances
    }

    # Convert the similarity weights into a list
    similarity_weights_list = [similarity_weights[team] for team in sorted(similarity_weights)]

    return similarity_weights_list

        
    



In [81]:
csv[csv['name'].str.contains('North Carolina')]

,Unnamed: 0,name,id
105,105,North Carolina,153
539,539,North Carolina Central,2428
549,549,North Carolina A&T,2448


# Player Stats

## Load Data

In [82]:
Team1 = 'North Carolina'
Team2 = 'Alabama'

##########################################

# Team 1

ids1 = get_id(Team1)[['name','id']]
idd1 =  (ids1.iloc[0,0], ids1.iloc[0,1])

df1 = all_games(idd1[0], idd1[1])
last1 = last_games(idd1[0], idd1[1])

team1, other_teams1 = set_up_data(idd1[1])

# Team 2

ids2 = get_id(Team2)[['name','id']]
idd2 =  (ids2.iloc[0,0], ids2.iloc[0,1])

df2 = all_games(idd2[0], idd2[1])
last2 = last_games(idd2[0], idd2[1])

team2, other_teams2 = set_up_data(idd2[1])


Done
Done
Done
Done


## Fit  KDE

In [83]:
# Team 1

weights1 = calculate_unbalanced_wasserstein(team2, other_teams1)

pt1 = df1[['game_id','Name', 'PTS', 'AST', 'REB']]
pt1 = pt1.pivot(index= 'game_id', columns = 'Name', values = ['PTS', 'AST', 'REB'])
pt1.fillna(0,inplace = True)

pt_kde1 = fitting_weights(pt1.values, weights1)

lpt1 = last1[['game_id','Name', 'PTS', 'AST', 'REB']]
lpt1 = lpt1.pivot(index= 'game_id', columns = 'Name', values = ['PTS', 'AST', 'REB'])
lpt1.fillna(0,inplace = True)

lpt_kde1 = fitting_weights(lpt1.values,weights1[-5:])

# Team 2

weights2 = calculate_unbalanced_wasserstein(team1, other_teams2)

pt2 = df2[['game_id','Name', 'PTS', 'AST', 'REB']]
pt2 = pt2.pivot(index= 'game_id', columns = 'Name', values = ['PTS', 'AST', 'REB'])
pt2.fillna(0,inplace = True)

pt_kde2 = fitting_weights(pt2.values, weights2)

lpt2 = last2[['game_id','Name', 'PTS', 'AST', 'REB']]
lpt2 = lpt2.pivot(index= 'game_id', columns = 'Name', values = ['PTS', 'AST', 'REB'])
lpt2.fillna(0,inplace = True)

lpt_kde2 = fitting_weights(lpt2.values, weights2[-5:])



C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:485: RuntimeWarning: divide by zero encountered in divide
  u = (a / Kv) ** fi_1
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:494: UserWarning: Numerical errors at iteration 0
  warnings.warn('Numerical errors at iteration %s' % i)
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:487: RuntimeWarning: divide by zero encountered in divide
  v = (b / Ktu) ** fi_2
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:485: RuntimeWarning: overflow encountered in divide
  u = (a / Kv) ** fi_1


Best bandwidth:  2.442053094548651
Best bandwidth:  2.6826957952797246


C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:485: RuntimeWarning: divide by zero encountered in divide
  u = (a / Kv) ** fi_1
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:494: UserWarning: Numerical errors at iteration 0
  warnings.warn('Numerical errors at iteration %s' % i)
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:485: RuntimeWarning: overflow encountered in divide
  u = (a / Kv) ** fi_1
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:487: RuntimeWarning: divide by zero encountered in divide
  v = (b / Ktu) ** fi_2
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:487: RuntimeWarning: overflow encountered in divide
  v = (b / Ktu) ** fi_2
C:\Users\timom\miniconda3\envs\snowflake\lib\site-packages\ot\unbalanced.py:494: UserWarning: Numerical errors at iteration 3
  warnings.warn('Numerical errors at iteration %s' % i)


Best bandwidth:  2.9470517025518097
Best bandwidth:  3.2374575428176433


In [85]:
weights2

[0.19973414929665925,
 0.19973414929665925,
 0.19973414929665925,
 0.19973414929665925,
 0.8664070210303106,
 0.5132192482662833,
 0.19973414929665925,
 0.19973414929665925,
 0.12234707672869048,
 0.22956744560332115,
 0.24177531759955484,
 0.19973414929665925,
 0.19973414929665925,
 0.739475534081631,
 0.19973414929665925,
 0.06230640529947245,
 0.19973414929665925,
 0.19712588431928096,
 0.19973414929665925,
 0.19973414929665925,
 0.528334107791975,
 0.19973414929665925,
 0.3004871282086729,
 0.29772864871654336,
 0.19973414929665925,
 0.19973414929665925,
 0.20698272602477413,
 0.7538327502153657,
 0.19973414929665925,
 1.0,
 0.19973414929665925,
 0.19973414929665925,
 0.19973414929665925,
 0.19973414929665925]

## Results

In [86]:
player_name = 'Estrada'

for a, b in zip(pt1.columns, pt2.columns):
    if player_name in a[1]:
        print('Team 1')
        break 
    elif player_name in b[1]:
        print('Team 2')
        break
        

Team 2


### Team 1

In [56]:
print(idd1[0])
print(list(pt1.columns))

San Diego State
[('PTS', 'BJ Davis'), ('PTS', 'Cade Alger'), ('PTS', 'Cam Lawin'), ('PTS', 'Darrion Trammell'), ('PTS', 'Demarshay Johnson Jr.'), ('PTS', 'Desai Lopez'), ('PTS', 'Elijah Saunders'), ('PTS', 'Jaedon LeDee'), ('PTS', 'Jay Pal'), ('PTS', 'Lamont Butler'), ('PTS', 'Micah Parrish'), ('PTS', 'Miles Byrd'), ('PTS', 'Miles Heide'), ('PTS', 'Reese Waters'), ('PTS', 'Ryan Schwarz'), ('AST', 'BJ Davis'), ('AST', 'Cade Alger'), ('AST', 'Cam Lawin'), ('AST', 'Darrion Trammell'), ('AST', 'Demarshay Johnson Jr.'), ('AST', 'Desai Lopez'), ('AST', 'Elijah Saunders'), ('AST', 'Jaedon LeDee'), ('AST', 'Jay Pal'), ('AST', 'Lamont Butler'), ('AST', 'Micah Parrish'), ('AST', 'Miles Byrd'), ('AST', 'Miles Heide'), ('AST', 'Reese Waters'), ('AST', 'Ryan Schwarz'), ('REB', 'BJ Davis'), ('REB', 'Cade Alger'), ('REB', 'Cam Lawin'), ('REB', 'Darrion Trammell'), ('REB', 'Demarshay Johnson Jr.'), ('REB', 'Desai Lopez'), ('REB', 'Elijah Saunders'), ('REB', 'Jaedon LeDee'), ('REB', 'Jay Pal'), ('REB',

In [113]:
# Team 1

player_name1 = 'RJ Davis'

column_pts = list(pt1.columns).index(('PTS',player_name1))
column_reb =  list(pt1.columns).index(('REB',player_name1))
column_ast =  list(pt1.columns).index(('AST',player_name1))

column_pts1 = list(lpt1.columns).index(('PTS',player_name1))
column_reb1 =  list(lpt1.columns).index(('REB',player_name1))
column_ast1 =  list(lpt1.columns).index(('AST',player_name1))

# POINTS 

x = 9.5

# REBOUNDS 

y = 8.5

# ASSITS

z = 4.5

# DEFINE FOR JOINT

x_values = [x,y]

column_indices = [column_pts, column_reb]
column_indices1 = [column_pts1, column_reb1]


##########################################
# Non Joint 
##########################################
print("Points")
prob_pts1, c1, c2 = estimate_probability(pt_kde1, x, column_pts)
print(f"{list(pt1.columns)[column_pts][1]} over {x} PTS is {round(prob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_pts1 , c1 , c2 = estimate_probability(lpt_kde1, x, column_pts1)
print(f"{list(lpt1.columns)[column_pts1][1]} over {x} PTS is {round(lprob_pts1,4)} {round(c1,4), round(c2,4)}")

print("\nRebounds")
prob_reb1, c1 , c2 = estimate_probability(pt_kde1, y, column_reb)
print(f"{list(pt1.columns)[column_reb][1]} over {y} REB is {round(prob_reb1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_reb1, c1 , c2 = estimate_probability(lpt_kde1, y, column_reb1)
print(f"{list(lpt1.columns)[column_reb1][1]} over {y} REB is {round(lprob_reb1,4)} {round(c1,4), round(c2,4)}")

print("\nAssits")
prob_ast1, c1 , c2 = estimate_probability(pt_kde1, z, column_ast)
print(f"{list(pt1.columns)[column_ast][1]} over {z} AST is {round(prob_ast1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_ast1, c1 , c2 = estimate_probability(lpt_kde1, z, column_ast1)
print(f"{list(lpt1.columns)[column_ast1][1]} over {z} AST is {round(lprob_ast1,4)} {round(c1,4), round(c2,4)}")



Points
RJ Davis over 9.5 PTS is 0.9517 (0.9474, 0.956)
Last 5 Games
RJ Davis over 9.5 PTS is 0.9999 (0.9996, 1.0001)

Rebounds
RJ Davis over 8.5 REB is 0.0564 (0.0519, 0.061)
Last 5 Games
RJ Davis over 8.5 REB is 0.0452 (0.0412, 0.0491)

Assits
RJ Davis over 4.5 AST is 0.3408 (0.3316, 0.35)
Last 5 Games
RJ Davis over 4.5 AST is 0.2623 (0.2545, 0.2702)


In [101]:
implied(-105)

0.5121951219512195

In [98]:
##########################################
# Joint 
##########################################

# 1: over
# 0: under
over_under = (1,1)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde1, x_values, column_indices, over_under)
print(f"Over Over is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde1, x_values, column_indices1, over_under)
print(f"Over Over is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (1,0)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde1, x_values, column_indices, over_under)
print(f"Over Under is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde1, x_values, column_indices1, over_under)
print(f"Over Under is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (0,1)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde1, x_values, column_indices, over_under)
print(f"Under Over is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde1, x_values, column_indices1, over_under)
print(f"Under Over is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (0,0)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde1, x_values, column_indices, over_under)
print(f"Under Under is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde1, x_values, column_indices1, over_under)
print(f"Under Under is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


Joint Probability
Over Over is 0.217 (0.2097, 0.2243)
Last 5 Games
Over Over is 0.2457 (0.2379, 0.2536)

Joint Probability
Over Under is 0.1408 (0.1345, 0.1471)
Last 5 Games
Over Under is 0.1762 (0.1693, 0.1831)

Joint Probability
Under Over is 0.301 (0.2919, 0.3101)
Last 5 Games
Under Over is 0.1821 (0.1743, 0.1898)

Joint Probability
Under Under is 0.3402 (0.332, 0.3484)
Last 5 Games
Under Under is 0.3967 (0.388, 0.4054)


In [109]:
# NEXT STEPS
df1[df1['Name'] == 'Harrison Ingram'][['REB','PTS']]

,REB,PTS
0,4,12
0,7,14
0,9,10
0,10,16
0,2,20
0,10,13
0,6,20
0,6,11
0,4,20
0,4,10


In [54]:
implied(-125)

0.5555555555555556

In [50]:
kelley(210,100, np.mean([0.33914,0.5765]))

19.963904761904764

### Team 2

In [87]:
print(idd2[0])
print(list(pt2.columns))

Alabama
[('PTS', 'Aaron Estrada'), ('PTS', 'Davin Cosby Jr.'), ('PTS', 'Grant Nelson'), ('PTS', 'Jarin Stevenson'), ('PTS', 'Kai Spears'), ('PTS', 'Latrell Wrightsell Jr.'), ('PTS', 'Mark Sears'), ('PTS', 'Max Scharnowski'), ('PTS', 'Mohamed Wague'), ('PTS', 'Mouhamed Dioubate'), ('PTS', 'Nick Pringle'), ('PTS', 'Rylan Griffen'), ('PTS', 'Sam Walters'), ('AST', 'Aaron Estrada'), ('AST', 'Davin Cosby Jr.'), ('AST', 'Grant Nelson'), ('AST', 'Jarin Stevenson'), ('AST', 'Kai Spears'), ('AST', 'Latrell Wrightsell Jr.'), ('AST', 'Mark Sears'), ('AST', 'Max Scharnowski'), ('AST', 'Mohamed Wague'), ('AST', 'Mouhamed Dioubate'), ('AST', 'Nick Pringle'), ('AST', 'Rylan Griffen'), ('AST', 'Sam Walters'), ('REB', 'Aaron Estrada'), ('REB', 'Davin Cosby Jr.'), ('REB', 'Grant Nelson'), ('REB', 'Jarin Stevenson'), ('REB', 'Kai Spears'), ('REB', 'Latrell Wrightsell Jr.'), ('REB', 'Mark Sears'), ('REB', 'Max Scharnowski'), ('REB', 'Mohamed Wague'), ('REB', 'Mouhamed Dioubate'), ('REB', 'Nick Pringle'), 

In [105]:
# Team  2
player_name2 =  'Grant Nelson'


column_pts = list(pt2.columns).index(('PTS',player_name2))
column_reb =  list(pt2.columns).index(('REB',player_name2))
column_ast =  list(pt2.columns).index(('AST',player_name2))

column_pts1 = list(lpt2.columns).index(('PTS',player_name2))
column_reb1 =  list(lpt2.columns).index(('REB',player_name2))
column_ast1 =  list(lpt2.columns).index(('AST',player_name2))

# POINTS 

x = 9.5

# REBOUNDS 

y = 5.5

# ASSITS

z = 3

# DEFINE FOR JOINT

x_values = [x,y]

column_indices = [column_pts, column_reb]
column_indices1 = [column_pts1, column_reb1]


##########################################
# Non Joint 
##########################################
print("Points")
prob_pts1, c1, c2 = estimate_probability(pt_kde2, x, column_pts)
print(f"{list(pt2.columns)[column_pts][1]} over {x} PTS is {round(prob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_pts1 , c1 , c2 = estimate_probability(lpt_kde2, x, column_pts1)
print(f"{list(lpt2.columns)[column_pts1][1]} over {x} PTS is {round(lprob_pts1,4)} {round(c1,4), round(c2,4)}")

print("\nRebounds")
prob_reb1, c1 , c2 = estimate_probability(pt_kde2, y, column_reb)
print(f"{list(pt2.columns)[column_reb][1]} over {y} REB is {round(prob_reb1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_reb1, c1 , c2 = estimate_probability(lpt_kde2, y, column_reb1)
print(f"{list(lpt2.columns)[column_reb1][1]} over {y} REB is {round(lprob_reb1,4)} {round(c1,4), round(c2,4)}")

print("\nAssits")
prob_ast1, c1 , c2 = estimate_probability(pt_kde2, z, column_ast)
print(f"{list(pt2.columns)[column_ast][1]} over {z} AST is {round(prob_ast1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
lprob_ast1, c1 , c2 = estimate_probability(lpt_kde2, z, column_ast1)
print(f"{list(lpt2.columns)[column_ast1][1]} over {z} AST is {round(lprob_ast1,4)} {round(c1,4), round(c2,4)}")



Points
Grant Nelson over 9.5 PTS is 0.6486 (0.6392, 0.658)
Last 5 Games
Grant Nelson over 9.5 PTS is 0.6147 (0.6051, 0.6243)

Rebounds
Grant Nelson over 5.5 REB is 0.4708 (0.4609, 0.4807)
Last 5 Games
Grant Nelson over 5.5 REB is 0.4648 (0.456, 0.4735)

Assits
Grant Nelson over 3 AST is 0.2991 (0.2902, 0.3081)
Last 5 Games
Grant Nelson over 3 AST is 0.1967 (0.189, 0.2044)


In [17]:
implied(-115)

0.5348837209302325

In [106]:
##########################################
# Joint 
##########################################

# 1: over
# 0: under
over_under = (1,1)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde2, x_values, column_indices, over_under)
print(f"Over Over is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde2, x_values, column_indices1, over_under)
print(f"Over Over is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (1,0)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde2, x_values, column_indices, over_under)
print(f"Over Under is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde2, x_values, column_indices1, over_under)
print(f"Over Under is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (0,1)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde2, x_values, column_indices, over_under)
print(f"Under Over is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde2, x_values, column_indices1, over_under)
print(f"Under Over is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


over_under = (0,0)

print("\nJoint Probability")
jprob_pts1, c1, c2 = estimate_joint_probability(pt_kde2, x_values, column_indices, over_under)
print(f"Under Under is {round(jprob_pts1,4)} {round(c1,4), round(c2,4)}")
print("Last 5 Games")
ljprob_pts1 , c1 , c2 = estimate_joint_probability(lpt_kde2, x_values, column_indices1, over_under)
print(f"Under Under is {round(ljprob_pts1,4)} {round(c1,4), round(c2,4)}")


Joint Probability
Over Over is 0.3406 (0.3313, 0.35)
Last 5 Games
Over Over is 0.3615 (0.3514, 0.3716)

Joint Probability
Over Under is 0.3091 (0.2999, 0.3182)
Last 5 Games
Over Under is 0.2549 (0.2465, 0.2633)

Joint Probability
Under Over is 0.1315 (0.125, 0.1381)
Last 5 Games
Under Over is 0.1035 (0.0975, 0.1094)

Joint Probability
Under Under is 0.2201 (0.2125, 0.2277)
Last 5 Games
Under Under is 0.281 (0.2721, 0.29)


In [62]:
np.mean([0.3117,0.17]) < np.mean([0.29,0.21])

0.25

In [ ]:
implied(-105)

In [65]:
kelley(284,100,0.29)

3.9999999999999982

In [108]:
df2[df2['Name']=='Harison Ingram'][['PTS','REB']]

,PTS,REB


# Team Stats

In [ ]:
# csv[csv['name'].str.contains('James Madison')]

In [ ]:
# proj1, proj2 = game_projection('401638610')

# team_box = team_boxscore('James Madison', '401638610')
# # past_team_boxscore(name, team_id)


In [ ]:
# df['FG'] = df['FG'].apply(lambda x: float(x.split('-')[0]) / float(x.split('-')[1]) if float(x.split('-')[1]) != 0 else 0)
# df['3PT'] = df['3PT'].apply(lambda x: float(x.split('-')[0]) / float(x.split('-')[1])  if float(x.split('-')[1]) != 0 else 0)
# df['FT'] = df['FT'].apply(lambda x: float(x.split('-')[0]) / float(x.split('-')[1])  if float(x.split('-')[1]) != 0 else 0)
# df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
# data = df.pivot(index= 'game_id', columns = 'Name', values = df.columns.values[2:])
# data.fillna(0,inplace = True)

In [ ]:
# def playbyplay(game_id = game_id):
#     plays_url = f"https://cdn.espn.com/core/mens-college-basketball/playbyplay?xhr=1&gameId={game_id}"
#     data = get_data(plays_url)
    
#     competitors = data['gamepackageJSON']['header']['competitions'][0]['competitors']
#     for comp in competitors:
#         if comp['homeAway'] == 'home':
#             home = [comp['team']['location'], comp['record'][0]['summary']]
#         if comp['homeAway'] == 'away':
#             away = [comp['team']['location'], comp['record'][0]['summary']]
            
#     timeline = []

#     for event in data['gamepackageJSON']['plays']:
#         if event['shootingPlay']:
#             timeline.append((event['period']['number'],event['clock']['displayValue'],event['homeScore'], event['awayScore']))

#     df = pd.DataFrame(timeline, columns = ['Half','Time',home[0],away[0]])
    
#     df['H-A'] = df.iloc[:,2] -  df.iloc[:,3]
    
#     df['mtm'] = df['H-A'] - df['H-A'].shift()
    
#     df['mtm1'] = df['mtm'].cumsum()

    
#     return df, home[1], away[1]


# def all_pbp(name, team_id):
#     game_ids = get_game_ids(team_id)
#     del game_ids[-1]
#     df = pd.DataFrame(columns = ['game_id', 'Name', 'MIN', 'FG', '3PT', 'FT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS'])
#     for game in game_ids:
#         sub = player_boxscore(name, game)
#         sub['game_id'] = game
#         df = pd.concat([df,sub])
#     print("Done")
#     return df

# def last_pbp(name, team_id):
#     game_ids = get_game_ids(team_id)
#     del game_ids[-1]
#     df = pd.DataFrame(columns = ['game_id', 'Name', 'MIN', 'FG', '3PT', 'FT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS'])
#     for game in game_ids:
#         sub = player_boxscore(name, game)
#         sub['game_id'] = game
#         df = pd.concat([df,sub])
#     print("Done")
#     return df

